# Efficienza delle funzioni ricorsive e funzioni ausiliarie

Consideriamo la funzione `palindroma_ricorsiva`, vista la lezione precedente.

In [9]:
def palindroma_ricorsiva(s):
    """
    Restituisce True se s è palindroma, False altrimenti
    """
    if len(s) <= 1:      # caso base
        return True
    elif s[0] != s[-1]:  # casi ricorsivi
        return False
    else:
        return palindroma_ricorsiva(s[1:len(s)-1])

Questa è invece la versione iterativa vista qualche tempo fa.

In [10]:
def palindroma_iterativa(s):
    for i in range(len(s) // 2):
        # confronta il carattere i-esimo a partire dall'inizio con il carattere
        # i-esimo a partire dalla fine
        if s[i] != s[- i - 1]:
            return False
    return True

Vorremmo confrontare le prestazioni di queste due funzioni. Un tale confronto andrebbe fatto con stringhe molto grandi, ma i limiti alla dimensione del call stack (pila dei recordi di attivazione) in Python ci limitano a stringhe di circa 6000 caratteri (perchè una palindroma di 6000 caratteri causa 3000 chiamate ricorsive prima di raggiungere il caso base, e 3000 è la dimensione massima del call stack in Python).

In [31]:
# La seguente istruzione fallisce
palindroma_ricorsiva("a" * 7000)

RecursionError: maximum recursion depth exceeded

Fortunatamente, è possibile aumentare la dimensione dell call stack in Python, con la funzione `setrecursionlimit` del modulo `sys`. Notare che impostare un limite troppo grande potrebbe causare un crash dovuto all'esaurimento della memoria.

In [32]:
# Fissa il limite a 1000000 per la dimensione del call stack

from sys import setrecursionlimit
setrecursionlimit(1_000_000)

In [33]:
# Adesso la stessa istruzione di prima ha successo
palindroma_ricorsiva("a" * 7000)

True

Proviamo quindi a vedere il tempo di esecuzione della versione ricorsiva e iterativa, prima con una stinga lunga 50.000, poi una lunga 100.000.

In [37]:
s1 = "a" * 50_000

In [43]:
%%timeit
palindroma_iterativa(s1)

6.65 ms ± 513 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [44]:
%%timeit
palindroma_ricorsiva(s1)

128 ms ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
s2 = "a" * 100_000

In [45]:
%%timeit
palindroma_iterativa(s2)

18.1 ms ± 4.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [46]:
%%timeit
palindroma_ricorsiva(s2)

434 ms ± 37.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Notiamo due cose:
  1. `palindroma_ricorsiva` è più lenta di `palindroma_iterativa`. Questo è inevitabile:  per far funzonare la ricorsione, l'interprete Python deve salvare varie informazioni e deve mantenere la pila dei record di attivazione.
  2. Raddoppiando la dimensione della stringa, il tempo che impiega  `palindroma_iterativa` raddoppia, ma quello che ci mette `palindroma_ricorsiva` quasi quadruplica. È un fenomeno che abbiamo visto quando abbiamo analizzato l'algoritmo di ordinamento per selezione: il termpo di esecuzione di `palindroma_ricorsiva` è quadratico rispetto alla lunghezza della stringa.

Il problema al punto 2 è causato dal fatto che la funzione  `palindroma_ricorsiva` , ogni volta che richiama se stessa, deve prendere la stringa in input è generare una nuova stringa in cui ha tolto il primo e l'ultimo carattere. Questa nuova stringa occuperà nuovo spazio in memoria, che dipenderà dalla sua lunghezza (approssimativamente un byte ogni carattere). Inoltre, la creazione di questa stringa richiede anch'essa tempo, perché quella vecchia va copiata in una nuova zona di memoria (tranne, ovviamente, i due caratteri che vogliamo eliminare).

Per risolvere questo problema modifichiamo la funzione `palindroma_ricorsiva` in modo che prenda due parametri aggiuntivi (`first` e `last`). L'idea è che la nuova funziona non controlla se tutta la stringa in input è palindroma, ma solo se la parte compresa tra le posizioni `first` e `last` lo è. In pratica, la coppia `first` e `last` rappresentano una *finestra* sulla stringa sulla quale focalizzarsi (ricorda in questo l'algoritmo di ricerca binaria). In questo modo, quando dobbiamo richiamare la funzione ricorsivamente, non è necessario estrarre un sottostringa, ma possiamo semplicemente modificare gli estremi di questa *finestra*.

In [47]:
def palindroma_ricorsiva_aux(s, first, last):
    """
    Restituisce True se la parte di s compresta tra la posizione first e la posizione
    last è palindroma, False altrimenti. Se first > last si intende che la parte
    compresa tra first e last è vuota.
    """
    if last <= first:          # caso base, la sottostringa che ci interessa ha lunghezza 0 oppure 1
        return True
    elif s[first] != s[last]:  # casi ricorsivi
        return False
    else:
        # invece di richiamare "palindroma_ricorsiva_aux" su una porzione di "s" la
        # richiamiamo su "s" stessa, ma restingiamo la finestra di osservazione eliminando
        # il carattere in posizione "first" e quello in posizione "last"
        return palindroma_ricorsiva_aux(s, first+1, last-1)

Quando richiamiamo `palindroma_ricorsiva_aux` dobbiamo ricordarci di fornire anche i parametri `first` e `last`, pari normalmente a `0` e alla lunghezza della stringa meno 1.

In [49]:
palindroma_ricorsiva_aux("osso",0, 3)

True

In [50]:
palindroma_ricorsiva_aux("attenta",0, 6)

False

Poiché usare `palindroma_ricorsiva_aux` in questo è scomodo, spesso si definisce una funzione addizionale che fa da *interfaccia* tra `palindroma_ricorsiva_aux` e il programmatore. Il programmatore chiama questa funzione di interfaccia fornendo solo i parametri veramente necessari, gli altri vengono calcolati automaticamente.

In [51]:
def palindroma_ricorsiva2(s):
    # richiamo la funzione palindroma_ricorsiva_aux con i valori opportuni
    return palindroma_ricorsiva_aux(s, 0, len(s)-1)

In [52]:
palindroma_ricorsiva2("osso")

True

In [53]:
palindroma_ricorsiva2("attenta")

False

Misuriamo i tempi di esecuzione della nuova versione.

In [54]:
%%timeit
palindroma_ricorsiva2(s1)

13.2 ms ± 995 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [56]:
%%timeit
palindroma_ricorsiva2(s2)

26.5 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Notiamo che:
  1. Anche `palindroma_ricorsiva2` è più lenta di `palindroma_iterativa`, ma solo del doppio. 
  2. Raddoppiando la dimensione della stringa in input, il tempo di esecuzione di `palindroma_ricorsiva2` raddoppia, comportandosi bene, come la funzione iterativa, e non come quella ricorsiva non ottimizzata iniziale.

# Le torri di Hanoi

Fin'ora abbiamo visto funzioni ricorsive che si potevano scrivere senza troppi problemi (anzi, spesso in maniera più intuitiva) in maniera ricorsiva. Ci sono però alcuni problemi che hanno soluzioni ricorsive molto semplici e soluzioni iterative estremamete complesse.

Consideriamo il caso del gioco delle [torri di Hanoi](https://it.wikipedia.org/wiki/Torre_di_Hanoi). Secondo Wikipedia:
<blockquote>
La Torre di Hanoi   è un rompicapo matematico composto da tre paletti e un certo numero di dischi di grandezza decrescente, che possono essere infilati in uno qualsiasi dei paletti. Il gioco inizia con tutti i dischi incolonnati su un paletto in ordine decrescente, in modo da formare un cono. Lo scopo del gioco è portare tutti i dischi su un paletto diverso, potendo spostare solo un disco alla volta e potendo mettere un disco solo su un altro disco più grande, mai su uno più piccolo. 
</blockquote>

Potete trovare una versione on-line del gioco su https://www.geogebra.org/m/fN74ZAsj

Il nostro obiettivo è scrivere una funzione che, preso un numero `n` in input, stampi l'elenco delle mosse da fare per spostare `n` dischi dal piolo n. 1 al piolo n. 3. Scrivere un programma che stampi questo elenco può sembrare estremamente complicato. Tuttavi, supponiamo che ci sia un nostro amico che sa come spostare `n-1` dischi da un piolo ad un altro. Allora, per spostare `n` dischi dal piolo 1 al piolo 3 possiamo procedere come segue:
  1. chiediamo all'amico di spostare `n-1` dischi  dal piolo 1 al 2;
  2. spostiamo l'ultimo disco rimastro nel piolo 1 (il disco più grande di tutti) al piolo 3;
  3. chiediamo di nuovo al nostro amico di spostare `n-1` dischi, questa volta dal piolo 2 al piolo 3.

Possiamo trasformare la sequenza di punti qui sopra nel caso ricorsivo di una funzione che risolve il nostro problema. Ma dobbiamo generalizzare la nostra funzione facendogli pendere due ulteriori parametri che sono il piolo di partenza e il piolo di arrivo dello spostamento. Come caso base consideriamo il problema in cui vogliamo spostare zero pioli: in questo caso, ovviamente, l'elenco degli spostamenti da compiere è vuoto.

In [58]:
def hanoi(n, from_peg, to_peg):
    """
    Stampa la sequenza di operazioni necessarie per spostare "n" dischi dal piolo "from_peg" al piolo "to_peg".
    I valori di "from_peg" e "to_peg" devono essere diversi tra di loro, e possono valere 1, 2 o 3.
    """
    if n == 0:  # caso base, nulla da stampare
        return
    # from_peg e to_peg sono i pioli di partenza e destinazione (devono essere diversi).
    # quando chiamiamo ricorsivamente la funzione hanoi, dobbimo spostare n-1 dischi dal
    # piolo from_peg ad un piolo temporaneo, e poi dal piolo temporaneo a to_peg. Il piolo
    # temporaneo è quello diverso sia da from_peg che da to_peg. L'istruzine if che segue
    # determina il piolo temporaneo.
    if from_peg != 1 and to_peg != 1:
        tmp_peg = 1
    elif from_peg != 2 and to_peg != 2:
        tmp_peg = 2
    else:
        tmp_peg = 3
    # prima chiamata ricorsiva
    hanoi(n-1, from_peg, tmp_peg)
    # stampa l'istruzione per spostare l'ultimo piolo da from_peg a to_peg
    print(from_peg, "->", to_peg)
    # seconda chiamata ricorsiva
    hanoi(n-1, tmp_peg, to_peg)

Poiché denotiamo i tre pioli con i numeri 1, 2 e 3, osserviamo che:
  1. la somma di questi tre numeri fa 6
  2. se calcoliamo `6 - from_peg - to_peg` otteniamo il piolo temporaneo da usare, al posto dell'if multiplo di prima.

Usando questo metodo per calcolare `tmp_peg` otteniamo la seguente.

In [60]:
def hanoi(n, from_peg, to_peg):
    """
    Stampa la sequenza di operazioni necessarie per spostare
    "n" dischi dal piolo "from_peg" al piolo "to_peg".
    """
    if n == 0:  # caso base, nulla da stampare
        return
    # determina il piolo temporaneo
    tmp_peg = 6 - from_peg - to_peg
    # prima chiamata ricorsiva
    hanoi(n-1, from_peg, tmp_peg)
    # stampa l'istruzione per spostare l'ultimo piolo da from_peg a to_peg
    print(from_peg, "->", to_peg)
    # seconda chiamata ricorsiva
    hanoi(n-1, tmp_peg, to_peg)

E queste sono le istruzioni per risolvere il problema delle torri di Hanoi con 4 dischi.

In [61]:
hanoi(4, 1, 3)

1 -> 2
1 -> 3
2 -> 3
1 -> 2
3 -> 1
3 -> 2
1 -> 2
1 -> 3
2 -> 3
2 -> 1
3 -> 1
2 -> 3
1 -> 2
1 -> 3
2 -> 3
